# Basic CNN

In [2]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as Data

In [3]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        #Image shape = (-1, 28, 28, 1)
        #Conv => (-1, 28, 28, 32)
        #output shape => (-1, 14, 14, 32)
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32,kernel_size = 3, stride= 1, padding = 1),
                                      nn.ReLU(), nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        #Conv => (-1, 14, 14, 64)
        #Pool => (-1, 7, 7, 64)
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                     nn.ReLU(), nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        #FC:  (7*7*64) inputs => (10) outputs
        self.fc = nn.Linear(7*7*64, 10, bias = True)
        nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, data):
        outputs = self.layer1(data)
        outputs = self.layer2(outputs)
        outputs = outputs.view(outputs.size(0), -1)
        outputs = self.fc(outputs)
        return outputs
        

In [4]:
#MNIST data
mnist_trainset = datasets.MNIST(root = './data',train =True,transform=transforms.ToTensor(),download=True)
mnist_testset = datasets.MNIST(root = './data', train = False, transform=transforms.ToTensor(),download=True)

#model
model = CNN()

#parameters
lr = 0.01
epochs = 10
batch_size = 100

#data
traindata = Data.DataLoader(dataset=mnist_trainset, batch_size=batch_size, shuffle=True)

#Optimizer / cost
optimizer = optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()  #Sotfmax is internally computed

In [6]:
for epoch in range(1, epochs +1):
    avg_cost = 0
    total_batch = len(mnist_trainset) // batch_size
    
    for i, (batch_xs, batch_ys) in enumerate(traindata):
        X = Variable(batch_xs)
        Y = Variable(batch_ys)
        
        optimizer.zero_grad()
        outputs = model.forward(X)
        cost = criterion(outputs, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost.data / total_batch
        
    print('Epoch {}, cost = {}'.format(epoch, avg_cost))

Epoch 1, cost = 0.17255689203739166
Epoch 2, cost = 0.06926210224628448
Epoch 3, cost = 0.060514241456985474
Epoch 4, cost = 0.053351029753685
Epoch 5, cost = 0.050637826323509216
Epoch 6, cost = 0.048927318304777145
Epoch 7, cost = 0.04560926556587219
Epoch 8, cost = 0.044997867196798325
Epoch 9, cost = 0.04215766862034798
Epoch 10, cost = 0.04083450883626938


In [7]:
#Test model and check accuracy
model.eval()  # set the model to evaluation mode (dropout = False)

X_test = Variable(mnist_testset.test_data.view(-1, 1, 28, 28)).float()  
Y_test = Variable(mnist_testset.test_labels)

prediction = model.forward(X_test)
correct = (prediction.argmax(1) == Y_test)
accuracy = correct.float().mean()
print('Accuracy: ', accuracy)

Accuracy:  tensor(0.9189)


# Deep CNN
* add 1 more convolutional layer and 2 more FC

In [25]:
class Deep_CNN(nn.Module):
    
    def __init__(self):
        super(Deep_CNN, self).__init__()
        #Image shape = (-1, 28, 28, 1)
        #Conv => (-1, 28, 28, 32)
        #output shape => (-1, 14, 14, 32)
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32,kernel_size = 3, stride= 1, padding = 1),
                                      nn.ReLU(), nn.MaxPool2d(kernel_size = 2, stride = 2),
                                       nn.Dropout(p = 1 - keep_prob))
        
        #Conv => (-1, 14, 14, 64)
        #Pool => (-1, 7, 7, 64)
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                     nn.ReLU(), nn.MaxPool2d(kernel_size = 2, stride = 2),
                                       nn.Dropout(p = 1 - keep_prob))
        
        #Conv => (-1, 7, 7, 128)
        #Pool => (-1, 4, 4, 128)
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
                                     nn.ReLU(), nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1),
                                       nn.Dropout(p = 1 - keep_prob))
        
        #L4:  (4*4*128) inputs => (625) outputs
        self.fc1 = nn.Linear(4*4*128, 625, bias = True)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = nn.Sequential(self.fc1, nn.ReLU(),
                                   nn.Dropout(p = 1-keep_prob))
        #L5  (625) inputs => 10 outputs
        self.fc2 = nn.Linear(625, 10, bias = True)
        nn.init.xavier_uniform_(self.fc2.weight)
            
    def forward(self, data):
        outputs = self.layer1(data)
        outputs = self.layer2(outputs)
        outputs = self.layer3(outputs)
        outputs = outputs.view(outputs.size(0), -1)
        outputs = self.layer4(outputs)
        outputs = self.fc2(outputs)
        return outputs
        

In [28]:
#model
keep_prob = 0.7
model = Deep_CNN()

#parameters
lr = 0.001
epochs = 10
batch_size = 100

#data
traindata = Data.DataLoader(dataset=mnist_trainset, batch_size=batch_size, shuffle=True)

#Optimizer / cost
optimizer = optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()  #Sotfmax is internally computed

In [29]:
for epoch in range(1, epochs +1):
    avg_cost = 0
    total_batch = len(mnist_trainset) // batch_size
    
    for i, (batch_xs, batch_ys) in enumerate(traindata):
        X = Variable(batch_xs)
        Y = Variable(batch_ys)
        
        optimizer.zero_grad()
        outputs = model.forward(X)
        cost = criterion(outputs, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost.data / total_batch
        
    print('Epoch {}, cost = {}'.format(epoch, avg_cost))

Epoch 1, cost = 0.27524930238723755
Epoch 2, cost = 0.0781298354268074
Epoch 3, cost = 0.06037244573235512
Epoch 4, cost = 0.04962151497602463
Epoch 5, cost = 0.0436650775372982
Epoch 6, cost = 0.040985699743032455
Epoch 7, cost = 0.03547627851366997
Epoch 8, cost = 0.0335809662938118
Epoch 9, cost = 0.03145352378487587
Epoch 10, cost = 0.03033491224050522


In [30]:
#Test model and check accuracy
model.eval()  # set the model to evaluation mode (dropout = False)

X_test = Variable(mnist_testset.test_data.view(-1, 1, 28, 28)).float()  
Y_test = Variable(mnist_testset.test_labels)

prediction = model.forward(X_test)
correct = (prediction.argmax(1) == Y_test)
accuracy = correct.float().mean()
print('Accuracy: ', accuracy)

Accuracy:  tensor(0.9543)
